# NARX and Recurrent Neural Networks
## Application to Ground Vibration Test of an F-16 aircraft. Description of the benchmark: https://www.nonlinearbenchmark.org/benchmarks/f-16-gvt

### Course on Deep Learning for System Identification
### Authors: Dario Piga, Marco Forgione
### Milano, July, 2025

## Goal: estimate mechanical vibrations on an F-16 aircraft in response to a shaker input

**- Inputs**: 
1. voltage measured at the output of the signal generator amplifier, acting as a reference input,
2. actual force provided by the shaker and measured by a impedance head at the excitation location.

**- Outputs:** acceleration signals measured:
1. at the excitation location ('Acceleration 1')
2. on the right wing next to the interface ('Acceleration 2')
3. on the payload next to the interface ('Acceleration 3')

# Exercise 1: NARX with Feedforward NNs:

- **Estimate a nonlinear model** describing the relation between the inputs and the three outputs.
- Since the two inputs are strongly correlated, you can use either both inputs in your model, or just one of them.
- Create three independent models, one for each output. Alternatively, create one single model with 3 outputs.
- Use a **fully-connected feedforward Neural Network** taking in past measurements of the two inputs (up to time k) and  measurements of the output (up to time k-1), and returning an estimate of the output at time k. For a single output and two inputs, the structure is:
$$y(k) = F(\overbrace{y(k-1), \dots, y(k-n_a), u_1(k), \dots, u_1(k - n_b + 1), u_2(k), \dots, u_2(k - n_b + 1)}^{=\phi(k)};\; W)$$
- **Test your results** on the dataset not used for training. Check the performance of your model both on 1-step ahead prediction and simulation. You can use the following metrics to assess the performance of your model:

\begin{align*}
\mathrm{RMSE} &= \sqrt{\frac{1}{T} \sum_{t=1}^T \big (y_t - \hat y_t \big)^2} \\
R^2 & = 1 - \frac{\sum_{t=1}^T \big (y_t - \hat y_t \big)^2}{\sum_{t=1}^T \big (y_t - \bar y_t \big)^2}
\end{align*}

## Step 0: Data ingestion and normalization (done)

Train and test datasets are saved in the GitHub folder *Dataset\F16*, both in a .mat and .csv  format.
- **Train dataset**: F16Data_SineSw_Level3
- **Test datasets**: F16Data_SineSw_Level4_Validation


In [1]:
import os
import pandas as pd
import torch
import numpy as np

In [2]:
folder = os.path.join("..", "..", "data", "F16")
filename_train = os.path.join(folder, "F16Data_SineSw_Level3.csv")
filename_test = os.path.join(folder, "F16Data_SineSw_Level4_Validation.csv")

In [3]:
df_train = pd.read_csv(filename_train)
df_test = pd.read_csv(filename_test)

In [4]:
# data normalization
ds_mean = df_train.mean()
ds_std = df_train.std()
df_train = df_train - ds_mean / ds_std
df_test = df_test - ds_mean / ds_std

In [5]:
df_train.head()

,Force,Voltage,Acceleration1,Acceleration2,Acceleration3,Fs,Unnamed: 6
0,-0.018484,0.000011,0.000699,-0.002362,0.002817,NaN,NaN
1,-0.014168,0.000006,0.003286,0.000966,-0.004438,NaN,NaN
2,0.076786,0.000006,-0.006074,0.002666,0.001021,NaN,NaN
3,0.008705,0.000021,-0.004311,0.002797,0.002421,NaN,NaN
4,0.014639,0.000009,0.000723,0.000128,-0.001134,NaN,NaN


## Step 1: Data Exploration (TODO). 

Familiarize with the data. It is also a good practive to plot the input and output signals, see the length, mean and standard deviation, etc.

## Step 2: Lagged dataset preparation (sketched)

Create a dataset suitable for NARX training containing past measurements of inputs and output, with instances in the form:

$$\bigg ( \overbrace{y(k-1), \dots, y(k-n_a), u_1(k), \dots, u_1(k - n_b + 1), u_2(k), \dots, u_2(k - n_b + 1)}^{=\phi(k)}, \;\;y(k) \bigg)$$

In [6]:
output_vars = ["Acceleration1"]
input_vars = ["Force", "Voltage"]
na = 3
nb = 20

You may use/adapt the following ``lag_dataframe`` utility function, or follow a different approach

In [7]:
def lag_dataframe(ds, lag_min, lag_max, columns=None, fill_value=np.nan):
    """
    Create a dataframe containing lagged versions of variables in the input dataframe.
    lag_min: minimum lag
    lag_max: maximum lag
    columns: list of columns to lag, if None all columns are lagged
    fill_value: value to fill for missing data corresponding to negative time stamps
    """
    lagged_dict = {}
    lagged_vars = []
    columns = columns if columns is not None else ds.columns
    for var in columns:
        for lag in range(lag_min, lag_max+1):
            lagged_var = f"{var}(k-{lag})" if lag > 0 else f"{var}(k)"  # 'var(k-i)
            lagged_dict[lagged_var] = ds[var].shift(lag, fill_value=fill_value).copy()
            lagged_vars.append(lagged_var)
    ds_lag = pd.DataFrame(lagged_dict)
    return ds_lag,lagged_vars

In [8]:
# usage of the lag_dataframe utility function
ds_lag, lagged_vars = lag_dataframe(df_train[["Force", "Voltage"]], lag_min=1, lag_max=3)
ds_lag.head()

,Force(k-1),Force(k-2),Force(k-3),Voltage(k-1),Voltage(k-2),Voltage(k-3)
0,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.018484,NaN,NaN,0.000011,NaN,NaN
2,-0.014168,-0.018484,NaN,0.000006,0.000011,NaN
3,0.076786,-0.014168,-0.018484,0.000006,0.000006,0.000011
4,0.008705,0.076786,-0.014168,0.000021,0.000006,0.000006


In [9]:
...

Ellipsis

Define PyTorch's Dataset and DataLoader

## Step 3: Network Definition (sketched). 

Define a class for your network (see template below)

In [10]:
# importing torch and other packages

import matplotlib.pyplot as plt
import torch 
import torch.nn as nn # the pytorch nn model contains all the layers we need to define a feedforward NN 
import torch.nn.functional as F
from torchsummary import summary
import torch.optim as optim
from sklearn.metrics import r2_score
import os
import pandas as pd
import numpy as np
  
# Enable interactive figures for Jupyter Notebooks.
%matplotlib widget 

In [11]:
class FeedforwardNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super().__init__()

        # define here your hidden layers (Linear Layers and activation functions) and output layer (Linear Layer)

        ...

    def forward(self, x):
        # Pass the input through the hidden layer, apply activation, and then pass through output layer

        ...

        return x


## Step 4: Training (TODO).
Train your network (see template below, complete it and/or modify it as you like)

## Step 5: Performance assessment (TODO) 

Assess performance on test dataset (it is also a good practice to first assess performance in the training dataset) 

Performance should be assessed in terms of one-step ahead prediction and simulation.

To quantify the performance, compute the RMSE, the $R^2$ coefficient, plot true output and estimate output.

# Exercise 2: Recurrent Neural Networks:

Repeat the same exercise, but **use an RNN network** (either Vanilla RNN or LSTM) instead of a feedforward network. This allows you to look multi-step ahead in your training, rather than than simple 1-step.

Hyper-parameter to select:
* structure of the network (namely, size of hidden state, number of layers)
* sub-sequence length for training
* optimization hyper-parameters (learning rate, maximum number of epochs)


## Data ingestion and normalization (done)

In [12]:
folder = os.path.join("..", "..", "data", "F16")
filename_train = os.path.join(folder, "F16Data_SineSw_Level3.csv")
filename_test = os.path.join(folder, "F16Data_SineSw_Level4_Validation.csv")

In [13]:
df_train = pd.read_csv(filename_train)
df_test = pd.read_csv(filename_test)

In [14]:
# data normalization
ds_mean = df_train.mean()
ds_std = df_train.std()
df_train = df_train - ds_mean / ds_std
df_test = df_test - ds_mean / ds_std

In [15]:
output_vars = ["Acceleration1"]
input_vars = ["Force", "Voltage"]
seq_len = 2000

In [16]:
df_train[input_vars]

,Force,Voltage
0,-0.018484,0.000011
1,-0.014168,0.000006
2,0.076786,0.000006
3,0.008705,0.000021
4,0.014639,0.000009
...,...,...
108472,-0.000034,0.000002
108473,-0.000034,0.000002
108474,-0.000034,0.000002
108475,-0.000034,0.000002


## Sequence DataSet preparation (sketched)

You may exploit the custom PyTorch Dataset defined in the pytorch_sysid.ipynb notebook

In [17]:
from torch.utils.data import Dataset

class SequenceDataset(Dataset):
    r"""A dataset returning sub-sequences extracted from longer sequences.
        For simplicity, this version does not support overlapping subsequences.
    Args:
        *tensors (Tensor): tensors that have the same size on the first dimension.
    Examples:
        >>> u = torch.randn(1000, 2) # 2 inputs
        >>> y = torch.randn(1000, 3) # 3 outputs
        >>> train_dataset = SequenceDataset(u, y, seq_len=100)
    """

    def __init__(self, *tensors, seq_len):
        self.tensors = tensors

        self.seq_len = seq_len
        assert all(tensor.shape[0] == self.tensors[0].shape[0] for tensor in self.tensors), "All tensors must have the same length"
        self.total_len = self.tensors[0].shape[0]

    def __len__(self):
        return int(self.total_len // self.seq_len)

    def __getitem__(self, idx):
        start = idx * self.seq_len
        stop = start + self.seq_len
        return [tensor[start:stop] for tensor in self.tensors]

In [18]:
# Datasets
train_ds = SequenceDataset(
    torch.Tensor(df_train[input_vars].values).type(torch.Tensor),
    torch.Tensor(df_train[output_vars].values).type(torch.Tensor),
    seq_len=seq_len
)

... # continue with the test dataset

Ellipsis